# Posmo DataMarket Python SDK

In [ ]:
import geopandas
import h3
import hvplot.pandas
import movingpandas

from datamap.posmo_datamarket_sdk import PosmoDataMarket

Configure your Posmo project and credentials.

In [ ]:
# Client configuration
project_code = <PROJECT_CODE>
username = <USERNAME>
password = <PASSWORD>

# Initialize client
datamarket = PosmoDataMarket(project_code=project_code, username=username, password=password)

# hvPlot configuration
hvplot_defaults = {'tiles': 'CartoLight', 'geo': True}

## Fetching data

You can fetch data by writing a PostgreSQL query with the `query` method.

You can open the database schema documentation with the `schema` method.
```python
datamarket.schema()
```

In [ ]:
# Fetch data
# Convert timestamp to nanoseconds for MovingPandas
response1 = datamarket.query('''
    SELECT json_build_object(
    	'type', 'Feature',
    	'geometry', ST_AsGeoJSON(the_geom)::json,
    	'properties', json_build_object(
    		'timestamp', timestamp * 1000000,
    		'featureCollectionId', feature_collection_id
    	)
    ) AS json
    FROM features;
''')
features1 = [row['json'] for row in response1]

## Visualization

You can visualize and analyze the data with your chosen frameworks.

In [ ]:
# Visualize with GeoPandas
geo_data_frame1 = geopandas.GeoDataFrame.from_features(features1, crs='epsg:4326')
geo_data_frame1.hvplot(**hvplot_defaults)

In [ ]:
# Visualize with MovingPandas
trajectory_collection1 = movingpandas.TrajectoryCollection(geo_data_frame1, 'featureCollectionId', t='timestamp')
trajectory_collection1.hvplot(**hvplot_defaults)

## H3 support

H3 cell indices can be used to perform additional analysis.

Read more about H3 https://h3geo.org

In [ ]:
# Fetch data
response2 = datamarket.query('''
    SELECT h3_9 AS h3, COUNT(id) AS count
    FROM features
    GROUP BY h3_9;
''')

# Transform H3 cell boundaries to valid GeoJSON
features2 = [
    {
        'type': 'Feature',
        'geometry': {
            'type': 'Polygon',
            'coordinates': [h3.h3_to_geo_boundary(row['h3'], geo_json=True)]
        },
        'properties': {
            'count': int(row['count'])
        }
    } for row in response2
]

# Visualize H3 cells
geo_data_frame2 = geopandas.GeoDataFrame.from_features(features2, crs='epsg:4326')
geo_data_frame2.hvplot.polygons(**hvplot_defaults, c='count', cmap='Reds', alpha=0.8)